In [ ]:
# Import library yang diperlukan
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
# Import Dataset Uji
transform_uji = pd.read_csv('./data_uji_clean.csv')
# Import Dataset WHO
who_stunting = pd.read_csv('./dataset-who-stunting.csv', sep = ';', decimal = ',')
who_wasting = pd.read_csv('./dataset-who-wasting.csv' , sep = ';', decimal = ',')

In [ ]:
# Ubah label jenis kelamin menjadi string nominal
transform_uji.loc[transform_uji.jenisKelamin == 'L', 'jenisKelamin'] = '0'
transform_uji.loc[transform_uji.jenisKelamin == 'P', 'jenisKelamin'] = '1'

# Ubah string nominal menjadi integer nominal
transform_uji.jenisKelamin = pd.to_numeric(transform_uji.jenisKelamin)

In [ ]:
# Buat kolom baru untuk label stunting dan wasting
transform_uji['statusStunting'] = 'Unknown'
transform_uji['statusWasting'] = 'Unknown'

In [ ]:
# Ubah nilai Gender pada dataset WHO stunting
who_stunting['Gender'] = who_stunting['Gender'].str.replace('Female', '1')
who_stunting['Gender'] = who_stunting['Gender'].str.replace('Male', '0')

# Format nilai Gender menjadi numeral
who_stunting['Gender'] = pd.to_numeric(who_stunting['Gender'])

In [ ]:
# Ubah nilai Gender pada dataset WHO wasting
who_wasting['Gender'] = who_wasting['Gender'].str.replace('Female', '1')
who_wasting['Gender'] = who_wasting['Gender'].str.replace('Male', '0')

# Format nilai Gender menjadi numeral
who_wasting['Gender'] = pd.to_numeric(who_wasting['Gender'])

In [ ]:
# Tentukan nilai statusStunting pada dataset uji
# Loop seluruh data pada dataset untuk mencari zscore dan mengubah statusStunting
for cursor_latih in transform_uji.itertuples():
    # Cari data LMS yang sesuai dengan data cursor_latih
    cursor_who = who_stunting.loc[(who_stunting.Month == cursor_latih.umur) & (who_stunting.Gender == cursor_latih.jenisKelamin)]
    # Kalkulasi nilai z-score stunting
    zscore = (((cursor_latih.tinggiBadan / cursor_who.M)**cursor_who.L)-1) / (cursor_who.L * cursor_who.S)
    # Ubah data statusStunting
    if zscore.item() < (-2):
        transform_uji.loc[transform_uji.index == cursor_latih.Index, 'statusStunting'] = 1
    else:
        transform_uji.loc[transform_uji.index == cursor_latih.Index, 'statusStunting'] = 0

In [ ]:
# Membuat dataframe baru untuk data istimewa
# istimewa disini yakni data yang tidak berkelipatan 0.5 pada dataset who-Wasting
data_istimewa = transform_uji.loc[transform_uji.tinggiBadan % 0.5 != 0, ['jenisKelamin', 'tinggiBadan']]

# Hapus duplikat pada iregular
data_istimewa.drop_duplicates(inplace=True)

In [ ]:
# Menyesuaikan nama kolom
data_istimewa.rename(columns ={'jenisKelamin' : 'Gender', 'tinggiBadan' : 'Length'}, inplace=True)

# Menggabungkan data pada dataframe istimewa dan dataframe WHO Weight-for-Height
new_who_wasting = pd.concat([who_wasting, data_istimewa])

In [ ]:
# Sortir data menurut Length
new_who_wasting.sort_values(by = ['Gender', 'Length'], inplace = True)

# Reset Index
new_who_wasting.reset_index(drop = True, inplace = True)

In [ ]:
# Menerapkan interpolasi kedalam dataframe
# limit_area=inside untuk mengubah hanya data istimewa saja
new_who_wasting.interpolate(limit_area = 'inside', inplace=True)

In [ ]:
# Tentukan nilai statusWasting pada dataset latih
# Loop seluruh data pada dataset untuk mencari zscore dan mengubah statusWasting
for cursor_latih in transform_uji.itertuples():
    # Cari data LMS yang sesuai dengan data cursor_latih
    cursor_who = new_who_wasting.loc[(new_who_wasting.Length == cursor_latih.tinggiBadan) & (new_who_wasting.Gender == cursor_latih.jenisKelamin)]
    # Kalkulasi nilai z-score stunting
    zscore = (((cursor_latih.beratBadan / cursor_who.M)**cursor_who.L)-1) / (cursor_who.L * cursor_who.S)
    # Ubah data statusStunting
    if zscore.item() < (-2):
        transform_uji.loc[transform_uji.index == cursor_latih.Index, 'statusWasting'] = 1
    else:
        transform_uji.loc[transform_uji.index == cursor_latih.Index, 'statusWasting'] = 0

In [ ]:
# Ubah format statusStunting dan statusWasting menjadi numeral
transform_uji.statusStunting = pd.to_numeric(transform_uji.statusStunting) 
transform_uji.statusWasting = pd.to_numeric(transform_uji.statusWasting)

In [ ]:
# Visualisasikan Data statusStunting
stunting = transform_uji.loc[transform_uji.statusStunting == 1, 'statusStunting'].count()
non_stunting = transform_uji.loc[transform_uji.statusStunting == 0, 'statusStunting'].count()

data = [stunting, non_stunting]
labels = ['Stunting', 'non-Stunting']
explode = [0.0, 0.075]

fig, ax = plt.subplots()
ax.pie(data, explode = explode, labels = labels, autopct = '%.1f%%')
ax.axis('equal')

plt.show()

In [ ]:
# Visualisasikan Data statusWasting
wasting = transform_uji.loc[transform_uji.statusWasting == 1, 'statusWasting'].count()
non_wasting = transform_uji.loc[transform_uji.statusWasting == 0, 'statusWasting'].count()

data = [wasting, non_wasting]
labels = ['Wasting', 'non-Wasting']
explode = [0.0, 0.075]

fig, ax = plt.subplots()
ax.pie(data, explode = explode, labels = labels, autopct = '%.1f%%')
ax.axis('equal')

plt.show()

In [ ]:
# Membuat kolom keduanya atau sehat
transform_uji['statusKeduanya'] = 'Unknown'
transform_uji['statusSehat'] = 'Unknown'

# Tentukan nilai data pada label statusKeduanya dan statusSehat
for cursor_latih in transform_uji.itertuples():
    # Cek status keduanya dan ubah menjadi string angka 1, bukan integer
    if cursor_latih.statusStunting == 1:
        if cursor_latih.statusWasting == 1:
            transform_uji.loc[transform_uji.index == cursor_latih.Index, 'statusKeduanya'] = '1'
            continue
    elif cursor_latih.statusStunting == 0:
        if cursor_latih.statusWasting == 0:
            transform_uji.loc[transform_uji.index == cursor_latih.Index, 'statusSehat'] = '1'
            continue
            
# Setelah selesai mengecek dan merubah status keduanya dan sehat
# Ganti kata Unknown menjadi string 0, bukan integer
transform_uji.statusKeduanya = transform_uji.statusKeduanya.str.replace('Unknown', '0')
transform_uji.statusSehat = transform_uji.statusSehat.str.replace('Unknown', '0')

# Format tipe data statusKeduanya dan statusSehat
transform_uji.statusKeduanya = pd.to_numeric(transform_uji.statusKeduanya)
transform_uji.statusSehat = pd.to_numeric(transform_uji.statusSehat)

In [ ]:
# Visualisasikan Data Keduanya, Sehat, dan Pengidap salah satu penyakit
dataKeduanya = transform_uji.loc[transform_uji.statusKeduanya == 1, 'statusKeduanya'].count()
dataSehat = transform_uji.loc[transform_uji.statusSehat == 1, 'statusSehat'].count()
dataPengidap = transform_uji.jenisKelamin.count() - (dataKeduanya + dataSehat)

data = [dataKeduanya, dataSehat, dataPengidap]
labels = ['Keduanya', 'Sehat', 'Stunting atau Wasting']
explode = [0.075, 0.075, 0.075]

fig1, ax1 = plt.subplots()
ax1.pie(data, explode = explode, autopct = '%.1f%%', labels = labels, shadow = True)
ax1.axis('equal')

plt.show()

In [ ]:
# Pembuatan label target statusBalita
transform_uji['statusBalita'] = 'Unknown'

# Menetapkan nilai statusBalita berdasar 4 fitur
for cursor_latih in transform_uji.itertuples():
    if cursor_latih.statusSehat == True:
        transform_uji.loc[transform_uji.index == cursor_latih.Index, 'statusBalita'] = 'Sehat'
    elif cursor_latih.statusKeduanya == True:
        transform_uji.loc[transform_uji.index == cursor_latih.Index, 'statusBalita'] = 'Keduanya'
    elif cursor_latih.statusWasting == True:
        transform_uji.loc[transform_uji.index == cursor_latih.Index, 'statusBalita'] = 'Wasting'
    elif cursor_latih.statusStunting == True:
        transform_uji.loc[transform_uji.index == cursor_latih.Index, 'statusBalita'] = 'Stunting'   

In [ ]:
# Visualisasi Persebaran Data
nilaiStunting = transform_uji.loc[transform_uji.statusBalita == 'Stunting', 'statusBalita'].count()
nilaiWasting = transform_uji.loc[transform_uji.statusBalita == 'Wasting', 'statusBalita'].count()
nilaiKeduanya = transform_uji.loc[transform_uji.statusBalita == 'Keduanya', 'statusBalita'].count()
nilaiSehat = transform_uji.loc[transform_uji.statusBalita == 'Sehat', 'statusBalita'].count()

data = [nilaiSehat, nilaiKeduanya, nilaiWasting, nilaiStunting]
explode = [0.1, 0.1, 0.1, 0.1]
labels = ['Sehat', 'Keduanya', 'Hanya Wasting', 'Hanya Stunting']

fig, ax = plt.subplots()

ax.pie(data, explode = explode, labels = labels, autopct = '%1.1f%%', shadow = True)
ax.axis('equal')

plt.show()

In [ ]:
# Buat kolom baru untuk data normalisasi fitur utama
transform_uji['nUmur'] = 'Unknown'
transform_uji['nBeratBadan'] = 'Unknown'
transform_uji['nTinggiBadan'] = 'Unknown'

# Normalisasi setiap fitur utama numeral
for cursor_latih in transform_uji.itertuples():
    normalisasi_umur = (cursor_latih.umur - transform_uji.umur.min()) / (transform_uji.umur.max() - transform_uji.umur.min())
    normalisasi_beratBadan = (cursor_latih.beratBadan - transform_uji.beratBadan.min()) / (transform_uji.beratBadan.max() - transform_uji.beratBadan.min())
    normalisasi_tinggiBadan = (cursor_latih.tinggiBadan - transform_uji.tinggiBadan.min()) / (transform_uji.tinggiBadan.max() - transform_uji.tinggiBadan.min())
    # Ganti nilai yang ada pada fitur utama dengan nilai normalisasi
    # wajib menggunakan pd.to_numeric untuk menhindari error value
    transform_uji.loc[transform_uji.index == cursor_latih.Index, 'nUmur'] = pd.to_numeric(normalisasi_umur)
    transform_uji.loc[transform_uji.index == cursor_latih.Index, 'nBeratBadan'] = pd.to_numeric(normalisasi_beratBadan)
    transform_uji.loc[transform_uji.index == cursor_latih.Index, 'nTinggiBadan'] = pd.to_numeric(normalisasi_tinggiBadan)

In [ ]:
# Ubah format tipe data normalisasi
transform_uji.nUmur = pd.to_numeric(transform_uji.nUmur)
transform_uji.nBeratBadan = pd.to_numeric(transform_uji.nBeratBadan)
transform_uji.nTinggiBadan = pd.to_numeric(transform_uji.nTinggiBadan)

In [ ]:
# Cek data latih
transform_uji.head(10)

In [ ]:
# Ekspor dataset latih yang sudah diubah
transform_uji.to_csv('./data_uji_transformed.csv', index=False)